## 學生類（36張）

In [1]:
import pandas as pd
from pathlib import Path
import re
import os

In [2]:
projRoot = Path.cwd().parents[0]
root_rawdata = projRoot.joinpath('data/raw')

In [3]:
filepaths = sorted(root_rawdata.rglob('學*.csv'))
print(filepaths)

[PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學1-1.正式學籍在學學生人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學1-2.正式學籍在學學生人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學10.日間學制修讀雙聯學制學生數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學11.學校辦理國際合作與交流資料表-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學12-1.新生(含境外生)註冊率-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學12-2.新生(含境外生)註冊率-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學12-3.新生(含境外生)註冊率-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學13-1.於學年底處於休學狀態之人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學13-2.於學年底處於休學狀態之人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學14-1.退學人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/raw/學14-2.退學人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac

### Stage 1 & Stage 2 ( '...' to np.nan, object to int, & aggregate to school level )

In [86]:
# Stage 1
import numpy as np

root_stage1 = projRoot.joinpath('data/stage1')
root_stage1.mkdir(exist_ok=True)

for fn in filepaths:
    df = pd.read_csv(fn, na_values=['...']) #把...變成np.na
    if ('休學' in f'{fn}') or ('退學' in f'{fn}'): #目前休退學人數統計表各自「以「系(所)」」為單位的表，人數加總不一致，所以刪掉
        df = df.drop(['在學學生數'], axis=1)

    int_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()
    int_columns = list(filter(lambda x: x not in ['學年度', '學校統計處代碼', '系所代碼'], int_columns)) #不讓這幾個變項被加總

    if '學期' in list(df.columns): #有分1、2學期的資料表，依學期做long 轉wide
        df = df.groupby(['學年度', '學校統計處代碼', '學期'])[int_columns].agg(lambda x: np.nan if np.all(pd.isna(x)) else x.sum())
        df.reset_index(inplace=True)
        df = df.pivot(index=['學年度', '學校統計處代碼'], columns= '學期', values= int_columns)
        df.columns = [f"{col[0]}_{col[1]}" for col in df.columns]
        df.reset_index(inplace=True)
        df = df.drop(['學期_1', '學期_2'], axis=1)
    elif '學年底' in f'{fn}': #「以學年底狀態統計」的表，視作是第2學期的數據，變項名稱加上後綴'_2'
        df = df.rename(columns={col: col+'_2' for col in df[int_columns]})
    csv_filename = f'{root_stage1}/stage1_{fn.name}'
    df.to_csv(csv_filename, index=False)
    

/var/folders/ty/15vk89y91j50b13mgsdgzxc80000gn/T/ipykernel_59116/2940381913.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fn, na_values=['...']) #把...變成np.na


In [5]:
filepaths_s1 = sorted(root_stage1.rglob('stage1_*.csv'))
print(filepaths_s1)

[PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學1-1.正式學籍在學學生人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學1-2.正式學籍在學學生人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學10.日間學制修讀雙聯學制學生數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學11.學校辦理國際合作與交流資料表-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學12-1.新生(含境外生)註冊率-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學12-2.新生(含境外生)註冊率-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學12-3.新生(含境外生)註冊率-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學13-1.於學年底處於休學狀態之人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學13-2.於學年底處於休學狀態之人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage1/stage1_學14-1.退學人數-以「系(所)」統計.csv'), PosixPath

In [87]:
# Stage 2 (aggregate至校-年層級資料)

root_stage2 = projRoot.joinpath('data/stage2')
root_stage2.mkdir(exist_ok=True)

for fn in filepaths_s1:
    df = pd.read_csv(fn)
    df[['學年度', '學校統計處代碼']] = df[['學年度', '學校統計處代碼']].astype('object')
    # columns to aggregate (those that are int or float)
    int_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()
    int_columns = list(filter(lambda x: x not in ['系所代碼'], int_columns))
    # aggregate
    df = df.groupby(['學年度', '學校統計處代碼'])[int_columns].agg(lambda x: np.nan if np.all(pd.isna(x)) else x.sum())
    cols_to_drop = [col for col in df.columns if '%' in col]
    df = df.drop(cols_to_drop, axis=1)
    df.reset_index(inplace=True)
    csv_filename = f'{root_stage2}/stage2_{fn.name[7:]}'
    df.to_csv(csv_filename, index=False)
    print(cols_to_drop)

[]
[]
[]
[]


/var/folders/ty/15vk89y91j50b13mgsdgzxc80000gn/T/ipykernel_59116/1294000751.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fn)


['當學年度新生註冊率(%)D=〔(C+E)/(A-B+E)〕＊100％']
['當學年度各學制新生註冊率(%)E=〔(C+D)/(A-B+D)〕*100％']
[]
[]
[]
[]
[]
[]
['前一學年度學生就學穩定率(%)(C)(C=B/A)']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['境外學位生數之在學比率(%)']
['外國學生數之在學比率(%)']
['僑生及港澳生數之在學比率(%)']
['大陸地區來臺學位生數之在學比率(%)']
[]
[]
[]
[]
[]
[]
[]
[]
[]


### Stage 3 ( long form data, all merged together )

In [12]:
filepaths_s2 = sorted(root_stage2.rglob('stage2_*.csv'))
print(filepaths_s2)

[PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學1-1.正式學籍在學學生人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學1-2.正式學籍在學學生人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學10.日間學制修讀雙聯學制學生數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學11.學校辦理國際合作與交流資料表-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學12-1.新生(含境外生)註冊率-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學12-2.新生(含境外生)註冊率-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學12-3.新生(含境外生)註冊率-以「校」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學13-1.於學年底處於休學狀態之人數-以「系(所)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學13-2.於學年底處於休學狀態之人數-以「校(含學制班別)」統計.csv'), PosixPath('/Users/issac/Desktop/Dagshub/HEDB/data/stage2/stage2_學14-1.退學人數-以「系(所)」統計.csv'), PosixPath

In [88]:
# Stage 3

root_stage3 = projRoot.joinpath('data/stage3')
root_stage3.mkdir(exist_ok=True)

dfs = {}
for fn in filepaths_s2:
    dfs[fn.name] = pd.read_csv(fn)
    dfs[fn.name][['學年度', '學校統計處代碼']] = dfs[fn.name][['學年度', '學校統計處代碼']].astype('string') #merge的時候，校統計處代碼要先統一長度
    dfs[fn.name]['學校統計處代碼'] = dfs[fn.name]['學校統計處代碼'].str.zfill(4)
    dfs[fn.name]['學年度'] = dfs[fn.name]['學年度'].str.zfill(3)

# merge all 
keys = list(dfs.keys())
merged_df = dfs[keys[0]]
for i in range(1, len(keys)):
    merged_df = pd.merge(merged_df, dfs[keys[i]], how='outer')


merged_df.to_csv(f'{root_stage3}/stage3_student.csv', index=False)